In [63]:
import pandas as pd
import time

In [66]:
def display_time(func):
    def wrapper(*args):
        t1 = time.time()
        func(*args)
        t2 = time.time()
        print('Time used is: {:.4}m'.format((t2 - t1) / 60))
    return wrapper

In [2]:
def convert_date(data):
    year = data.split('/')[0]
    month = data.split('/')[1]
    
    if month == 'Jan' or month == 'Feb' or month == 'Mar':
        return year + ' Q1'
    elif month == 'Apr' or month == 'May' or month == 'Jun':
        return year + ' Q2'
    elif month == 'Jul' or month == 'Aug' or month == 'Sep':
        return year +  ' Q3'
    else:
        return year + ' Q4'

In [90]:
@display_time
def transform_data(nsw):
    year = nsw.columns[2].split(' ')[0]
    
    nsw = nsw.set_index(['Incident suburb', 'Offence'])
    nsw = nsw.stack().to_frame()
    nsw = nsw.rename(columns={0:'records'})
    nsw.index.names=['suburb', 'category', 'year quarter']
    nsw = nsw.groupby(['suburb', 'category', 'year quarter']).sum()
    nsw = nsw.reset_index()
    
    nsw['district'] = ''
    nsw['state_code'] = 'NSW'
    nsw['postcode'] = ''
    nsw['subcategory'] = ''
    
    nsw['year'] = nsw['year quarter'].str.split(' ').str[0]
    nsw['quarter'] = nsw['year quarter'].str.split(' ').str[1]
    nsw.drop(columns=['year quarter'], inplace=True)
    
    
    nsw = nsw[['district', 'suburb', 'postcode', 'state_code', 'category', 'subcategory', 'year', 'quarter', 'records']]
    
    nsw.to_excel('finished/NSW/NSW' + year + '.xlsx', index=False)
    print('NSW_', year, ' has finished!')

In [3]:
nsw = pd.read_excel('NSW.xlsx', header=None)
nsw.head(2)

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,Incident suburb,Offence,2011/Jan,2011/Feb,2011/Mar,2011/Apr,2011/May,2011/Jun,2011/Jul,2011/Aug,...,2015/Mar,2015/Apr,2015/May,2015/Jun,2015/Jul,2015/Aug,2015/Sep,2015/Oct,2015/Nov,2015/Dec
1,Unknown / missing,Murder*,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
for i in range(len(nsw.loc[0])):
    if i < 2:
        continue
    else:
        nsw.loc[0][i] = convert_date(nsw.loc[0][i])
               

In [5]:
nsw.drop(nsw[nsw[0] == 'Unknown / missing'].index,inplace=True)

In [6]:
columns_name_list = list(nsw.loc[0, :])

In [7]:
nsw.columns = columns_name_list

In [8]:
nsw.drop(nsw.loc[0:1,:].index, inplace=True)

In [9]:
nsw = nsw.reset_index().drop(columns=['index'])

In [49]:
nsw2011 = nsw[['Incident suburb', 'Offence', '2011 Q1', '2011 Q2', '2011 Q3', '2011 Q4']]
nsw2012 = nsw[['Incident suburb', 'Offence', '2012 Q1', '2012 Q2', '2012 Q3', '2012 Q4']]
nsw2013 = nsw[['Incident suburb', 'Offence', '2013 Q1', '2013 Q2', '2013 Q3', '2013 Q4']]
nsw2014 = nsw[['Incident suburb', 'Offence', '2014 Q1', '2014 Q2', '2014 Q3', '2014 Q4']]
nsw2015 = nsw[['Incident suburb', 'Offence', '2015 Q1', '2015 Q2', '2015 Q3', '2015 Q4']]

In [54]:
nsw_list = [nsw2011, nsw2012, nsw2013, nsw2014, nsw2015]

In [91]:
for i in range(len(nsw_list)):
    transform_data(nsw_list[i])

NSW_ 2011  has finished!
Time used is: 3.882m
NSW_ 2012  has finished!
Time used is: 3.895m
NSW_ 2013  has finished!
Time used is: 3.925m
NSW_ 2014  has finished!
Time used is: 3.874m
NSW_ 2015  has finished!
Time used is: 4.001m
